# Imports

In [ ]:
import os
import pandas as pd
import numpoy as np

# load data func

In [2]:
def load_data(folder_path):
  '''
  takes in directory path to orders, members and products .csv files,
  loads data and returns respective dataframes

  df_orders, df_members, df_products = src.load_data(folder_path)
  '''
  # list all files in folder_path 
  listdir = os.listdir(folder_path)
  print(listdir)

  # read csv and create dataframes
  for csv in listdir:
    if 'orders' in csv:
      df_orders = pd.read_csv(folder_path + csv, 
                              index_col='order_ID',
                              parse_dates=['delivery_date', 'created_at', 'updated_at'])
    elif 'members' in csv:
      df_members = pd.read_csv(folder_path + csv, 
                              #index_col=['member_ID', 'order_ID']
                              )
    elif 'products' in csv:
      df_products = pd.read_csv(folder_path + csv, 
                                index_col=['order_ID', 'product_ID']
                                )
  return df_orders, df_members, df_products

# features.py

In [3]:
def order_request_value(df_members, df_products):
  '''calculates order value for each order_request by a member
  
  takes in df_members and df_products,
  calculates order_request_value,
  and returns df_members with a column "order_request_value"
  '''

  # calculate order value for each member
  df_members['order_request_value'] = np.nan

  # step 1: loop through df_members to get 'order_ID' and 'order_requests'
  for i in df_members.index:
    order_ID = df_members.order_ID[i]
    order_request = eval(df_members.order_requests[i]) # eval() turns order_request string to dict
    
    # step 2: get 'product_ID' and 'filled_amount' from order_request
    order_request_df = pd.DataFrame(order_request, index=['filled', 'ordered']).T.reset_index() # setting index to prevent issues with missing dict keys 
    product_ID = order_request_df['index']
    filled_amount = order_request_df['filled'].replace(',', '.', regex=True) # str replace instances with ','
    filled_amount = filled_amount.replace(r'^\s*$', 0, regex=True).astype('float') # str replace empty strings

    # step 3: access net_price from df_products and calculate order_request_value
    net_price = df_products.loc[(order_ID, product_ID), 'net_price'].reset_index(drop=True)
    order_request_value = np.sum(net_price * filled_amount).round(2)
    
    # step 4: assign order_request_value to df_members
    df_members.loc[i , 'order_request_value'] = order_request_value

  return df_members

In [4]:
def net_total_price(df_products):
  # create column 'net_total_price' (net_price * amount_ordered) for each product
  df_products['net_total_price'] = df_products.net_price * df_products.amount_ordered
  return df_products

In [5]:
def delivery_date_to_members(df_members, df_orders):
  '''takes in df_members and df_orders
  and merges delivery_date to members dataframe
  
  returns df_members
  '''
  # adding delivery_dates to members DF
  df_members = df_members.merge(df_orders['delivery_date'].reset_index(), # reset_index since order_ID is in index
                                how='left',
                                on='order_ID'
                                )
  return df_members

In [6]:
def total_order_value(df_orders, df_products):
  # get total_order_value
  total_order_value = df_products.groupby('order_ID')['net_total_price'].sum()
  df_orders['total_order_value'] = total_order_value.round(2)
  return df_orders

def num_participating_members(df_orders, df_products):
  # get num_participating_members
  df_orders['num_participating_members'] = df_members.groupby('order_ID').size()
  return df_orders

# Load data

In [7]:
# set folder path
folder_path = '/content/drive/My Drive/DataScience/supercoop/supercoapp-data-analysis/data/processed/'
# load data
df_orders, df_members, df_products = load_data(folder_path)

['20201203_scoop_orders.csv', '20201203_scoop_members.csv', '20201203_scoop_products.csv']


# test functions

In [8]:
df_members.head(2)

,member_ID,order_ID,name,collected?,order_requests,deposits
0,1,1,XXX,True,"{'3': {'filled': '1.0', 'ordered': '1.0'}, '25...",NaN
1,2,1,XXX,True,"{'39': {'filled': '1.0', 'ordered': '1.0'}, '6...",NaN


In [9]:
df_products.head(2)

name  ... bundles_ordered
order_ID product_ID                                                ...                
1        3                  Artischockenherzen geviertelt in Lake  ...             1.0
         4           Tomaten getrocknet in Olivenöl, extra saftig  ...             1.0

[2 rows x 12 columns]

In [10]:
# order_request_value
df_members = order_request_value(df_members, df_products)

In [ ]:
def get_product_info(order_ID, product_ID):
  prod_info = df_products.loc[(order_ID, product_ID)]
  return prod_info

type(get_product_info(1, '3'))

pandas.core.series.Series

## order requests

In [36]:
def process_order_requests(df_members, df_products):

  output = pd.DataFrame()

  for i in df_members.index:
    # eval() turns order_request string to dict
    order_request = eval(df_members.order_requests[i]) 
    
    # step 2: turn order_requests into dataframe
    df = pd.DataFrame(order_request, index=['filled', 'ordered']).T
    df = df.rename_axis('product_ID').reset_index() # setting index to prevent issues with missing dict keys 

    # add order_ID and member_ID
    df['order_ID'] = df_members.order_ID[i]
    df['member_ID'] = df_members.member_ID[i]

    # concat order requests
    output = pd.concat([output, df], axis=0)

  # reorder columns
  reordered_columns = ['order_ID', 'member_ID', 'product_ID', 'filled', 'ordered']
  output = output[reordered_columns]

  # clean up data
  output['filled'] = output['filled'].replace(',', '.', regex=True) # str replace instances with ','
  output['filled'] = output['filled'].replace(r'^\s*$', 0, regex=True).astype('float') # str replace empty strings

  # merge product_info into output df
  cols_to_merge = ['name', 'unit', 'origin', 'category', 'producer', 'net_price', 'supplier_code']

  output = output.merge(df_products[cols_to_merge],
                how='left', 
                on=['order_ID', 'product_ID'])
  
  return output

In [37]:
order_requests = process_order_requests(df_members, df_products)
order_requests

,order_ID,member_ID,product_ID,filled,ordered,name,unit,origin,category,producer,net_price,supplier_code
0,1,1,3,1.0,1.0,Artischockenherzen geviertelt in Lake,200g,Italien,Konserven und Gläser,NaN,2.99,1204060
1,1,1,25,3.0,3.0,Soja-Schnetzel fein,125g,Frankreich,vegane Produkte,NaN,1.06,704565
2,1,1,27,2.0,2.0,Birnen-Apfel-Kraut,250g,EU,Aufstriche – fruchtiges,NaN,2.16,1500645
3,1,1,37,2.0,2.0,Quinoa bunt HIH,250g,nEU,Getreideprodukte,NaN,1.60,401990
4,1,1,39,2.0,2.0,Boulgour,500g,Italien,Getreideprodukte,NaN,1.68,420130
...,...,...,...,...,...,...,...,...,...,...,...,...
14374,194,62,121,1.0,1.0,"Gemischte Oliven mit Kräutern, ohne Stein geölt",170g,Griechenland,Konserven und Gläser,NaN,3.16,1202890
14375,194,62,182,1.0,1.0,Kokosblütenzucker,250g,Indonesien,Backzutaten,NaN,2.36,1400860
14376,194,62,268,1.0,1.0,streich`s drauf Basitom - (Basilikum / Tomate),160g,EU/nEU,Aufstriche – herzhaft,NaN,1.63,5100410
14377,194,62,990,1.0,1.0,Tahin (Sesammus) 250g,g,nEU,Aufstriche – nussiges,Rapunzel,2.35,170100


# analysis

## new_products

In [63]:
# string contains new product
order_requests[order_requests.product_ID.str.contains('new_product')]

,order_ID,member_ID,product_ID,filled,ordered,name,unit,origin,category,producer,net_price,supplier_code,net_total_price
106,1,31,new_product_1,1.0,NaN,Spielwaren originalbefüllt,NaN,NaN,NaN,NaN,21.60,NaN,21.60
107,1,31,new_product_2,1.0,NaN,Holzspielwaren im Thekenkorb,NaN,NaN,NaN,NaN,21.60,NaN,21.60
108,1,31,new_product_3,1.0,NaN,Spielwaren im Metallkörbchen,NaN,NaN,NaN,NaN,11.00,NaN,11.00
621,6,1,new_product_3,1.0,NaN,Schokolade Cafe Chavalo,1,NaN,NaN,NaN,2.32,NaN,2.32
622,6,1,new_product_8,1.0,NaN,Cafe Chavalo Espresso,1Kg,NaN,NaN,NaN,16.20,NaN,16.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14155,192,58,new_product_1,6.0,NaN,Eier Größe M (lose),Stück,NaN,NaN,NaN,0.33,NaN,1.98
14156,192,60,new_product_19,10.0,NaN,Eier M,Stück,NaN,NaN,NaN,0.33,NaN,3.30
14157,192,63,new_product_1,3.0,NaN,Eier Größe M (lose),Stück,NaN,NaN,NaN,0.33,NaN,0.99
14158,192,63,new_product_9,1.0,NaN,demeter Sesam Vollkorn Knäckebrot,1 Paket,NaN,NaN,NaN,1.54,NaN,1.54


In [68]:
#drop rows with new_product
order_requests_copy = order_requests.copy()
order_requests_clean = order_requests_copy[order_requests.product_ID.str.contains('new_product') == False]

order_requests_clean

In [92]:
print(f'droped {1 - (order_requests_clean.shape[0]/order_requests.shape[0]):.2f} % of products')

droped 0.04 % of products


## calculations

In [39]:
# calculating net_total_price
order_requests['net_total_price'] = order_requests.net_price * order_requests.filled
# change dtype of ordered
order_requests.ordered = order_requests.ordered.astype('float')

In [40]:
order_requests.head(2)

,order_ID,member_ID,product_ID,filled,ordered,name,unit,origin,category,producer,net_price,supplier_code,net_total_price
0,1,1,3,1.0,1.0,Artischockenherzen geviertelt in Lake,200g,Italien,Konserven und Gläser,NaN,2.99,1204060,2.99
1,1,1,25,3.0,3.0,Soja-Schnetzel fein,125g,Frankreich,vegane Produkte,NaN,1.06,704565,3.18


In [24]:
# check if filled same amount as ordered
order_requests[['filled', 'ordered']].sum()

filled     21696.67
ordered    23361.77
dtype: float64

In [28]:
# check category
order_requests.category.value_counts().head(10)

Gemüse                     4700
Obst                       1623
Milchprodukte – Käse        859
Konserven und Gläser        767
Eier                        437
Brot                        396
Fleisch & Wurst             392
Milchprodukte – Joghurt     348
Milchprodukte – Milch       330
Aufstriche – herzhaft       269
Name: category, dtype: int64

In [30]:
# check origin
order_requests.origin.value_counts().head(12)

Deutschland                        3424
regional                           2687
Italien                            1905
Spanien                            1473
EU/nEU                              599
Dominikanische Republik             271
Regional                            244
Görlitzer Brotschmiede              215
Frankreich                          161
Alt-Treptow                         154
Schweiz                             151
Görlitzer Brotschmiede, demeter     148
Name: origin, dtype: int64

In [41]:
# check supplier_code
order_requests.supplier_code.value_counts()

Lebenskunst    1067
Lindenhof       438
129121          391
127365          258
116405          244
               ... 
DE3095            1
215               1
190130            1
1890              1
1952              1
Name: supplier_code, Length: 653, dtype: int64

# members summary

In [94]:
# check product_ID variety
members_summary = pd.DataFrame()
# nunique products
members_summary['nunique_products'] = order_requests_clean.groupby('member_ID').product_ID.nunique()
# nunique categories
members_summary['nunique_categories'] = order_requests_clean.groupby('member_ID').category.nunique()

In [102]:
# net total price per order
members_summary['avg_order_amount'] = order_requests.groupby('member_ID').net_total_price.sum() / order_requests.groupby('member_ID').order_ID.nunique()

In [101]:
order_requests.groupby('member_ID').order_ID.nunique()

member_ID
1     106
2      84
3      87
4       2
6      96
7      78
9      77
10     59
11     57
12     53
13     82
17     92
19     50
20     22
21      1
22    105
23      7
30     94
31      1
33     79
34     74
35     66
36      1
38     60
39     78
40      2
41     79
42     23
43      6
44     80
45     36
46     80
47     73
48     10
49     10
50     75
51     63
52     32
54     32
55     25
56     13
57     10
58      9
60      9
61      4
62      5
63      8
Name: order_ID, dtype: int64

In [103]:
members_summary

,nunique_products,nunique_categories,avg_order_amount
member_ID,,,
1,410,51,30.894401
2,169,41,9.466224
3,315,46,24.584324
4,5,5,7.680000
6,247,34,23.208989
7,105,26,11.653668
9,230,40,30.127087
10,211,40,11.942832
11,146,35,14.386223
